In [ ]:
import pickle
import numpy as np
import pandas as pd
import pygeohash as pgh
import matplotlib.pyplot as plt
import reverse_geocoder as rg

from datetime import datetime
from tqdm import tqdm_notebook
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=4, progress_bar=True, use_memory_fs=False)

# Detecting Seasonal Patterns in User Movement

There is more to human movement than periodic patterns and social-based movement. While the paper accurately predicts human movement throughout the week, we propose a more all-encompassing analysis that paints a broader picture, exploring monthly and seasonal movement. To do so, we will use the original check-in and friendship datasets from Gowalla and Brightkite and separate them into subsets representing a narrower time of the year. We want to see what mobility patterns are exhibited in different countries and throughout different times of the year: users can go on vacation, travel abroad, and change homes. This would allow us to understand the travelling patterns of users around the world (i.e. who is more likely to travel and where) as well as the distribution of check-ins and their density. We want to produce visualizations showcasing these patterns on a world map. 

Our starting point is the first replication of the Friendship and Mobility paper. Below, the data is imported and cleaned

## Loading the data

In [ ]:
# The datasets have no headers, so we have to name them for clarity
checkin_header = ['user', 'checkin_time', 'latitude', 'longitude', 'location_id']
edges_header = ['user1', 'user2']

# Load the data by specifying the correct compression algorithm, separator and column names
checkin_brightkite_orig = pd.read_csv('data/loc-brightkite_totalCheckins.txt.gz', compression = 'gzip', sep = '\t', names = checkin_header)
edges_brightkite_orig = pd.read_csv('data/loc-brightkite_edges.txt.gz', compression = 'gzip', sep = '\t', names = edges_header)
checkin_gowalla_orig = pd.read_csv('data/loc-gowalla_totalCheckins.txt.gz', compression = 'gzip', sep = '\t', names = checkin_header)
edges_gowalla_orig = pd.read_csv('data/loc-gowalla_edges.txt.gz', compression = 'gzip', sep = '\t', names = edges_header)

## Cleaning the data

Some of the latitude and longitude values are not in the valid range of [-90, 90] and [-180, 180] respectively. Moreover, the location having (lat, long) = (0,0) is not a valid one . In both datasets, we remove rows in which either of the location attributes are not valid. We also remove the rows of the Brightkite dataset that have NaN values. 

In [ ]:
def clean(checkins):
    """ Rids the check-in data of invalid coordinates """
    to_drop = []
    # Select indexes to drop depending on conditions mentionned above 
    to_drop.append(checkins[(checkins['latitude'] < -90.0) | 
                                   (checkins['latitude'] > 90.0)].index)
    to_drop.append(checkins[(checkins['longitude'] < -180.0) | 
                                   (checkins['longitude'] > 180.0)].index)
    to_drop.append(checkins[(checkins['latitude'] == 0) & 
                                   (checkins['longitude'] == 0)].index)
    
    for item in to_drop:
        checkins.drop(item, inplace = True)
    
    return to_drop

In [ ]:
# Pass copy of DataFrama in order not to have to reload the original one in case of a mistake
checkin_gowalla    = clean(checkin_gowalla_orig.copy())
checkin_brightkite = clean(checkin_brightkite_orig.copy())

## Adding country information to check-ins

For each check-in, we append the country code corresponding to the country the check-in was made in. This is done using a reverse geohashing library [`reverse_geocoder`](https://github.com/thampiman/reverse-geocoder) that maps coordinates (latitutde, longitude) to information about a specific place (e.g. city, country, country code).

In [ ]:
def extract_coordinates(row):
    return (row['latitude'], row['longitude'])

def coordinates_to_country(df):
    """
        Maps the coordinates of a dataframe to the country
        code using reverse geocoding
    """
    coordinates_tuples = list(df.parallel_apply(extract_coordinates, axis = 1))
    geocodes = rg.search(coordinates_tuples)
    return [gc['cc'] for gc in geocodes]

In [ ]:
# Append country codes to each check-in for both datasets
checkin_brightkite['cc'] = coordinates_to_country(checkin_brightkite)
checkin_gowalla['cc']    = coordinates_to_country(checkin_gowalla)

checkin_brightkite.head()

## Determining the location of user homes

Here, the geohash is used to determine the home location for every user. Since this is similar to what was done in the replication, we will not go into detail about the implementation and justification.

In [ ]:
def geohash_encode(row):
    """
        Computes and returns the geohash with 
        precision 4 for a given row 
    """
    return pgh.encode(*extract_coordinates(row), precision=4)

def find_home(checkins, user_id):
    """
        Given a user id, finds the most common world cell and 
        computes average location to estimate home location
    """
    # Retrieve all the user's check-ins
    group = checkins[checkins['user']  == user_id]
    # Get the most common geohash for the user
    geohash = group.groupby('geohash').size().idxmax()
    
    # Keep all checkins that happened in the most common geohash and compute average position
    return group[group['geohash'] == geohash][['latitude', 'longitude']].mean()

def find_home_brightkite(user_id):
    return find_home(checkin_brightkite, user_id)

def find_home_gowalla(user_id):
    return find_home(checkin_gowalla, user_id)

In [ ]:
# Compute the geohash for each check-in entry: that will determine the cell in which each check-in belongs
checkin_brightkite['geohash'] = checkin_brightkite.parallel_apply(geohash_encode, axis = 1)
checkin_gowalla['geohash']    = checkin_gowalla.parallel_apply(geohash_encode, axis = 1)

In [ ]:
# All the users that made at least 1 check-in
brightkite_users = set(checkin_brightkite.user)
gowalla_users = set(checkin_gowalla.user)

# Create empty DataFrame with user ids and apply function to all users
brightkite_homes = pd.DataFrame(index = brightkite_users).index.to_series() \
    .parallel_apply(find_home_brightkite) 

gowalla_homes = pd.DataFrame(index = gowalla_users).index.to_series() \
    .parallel_apply(find_home_gowalla) 

# Add the country codes to the homes
brightkite_homes['cc'] = coordinates_to_country(brightkite_homes)
gowalla_homes['cc'] = coordinates_to_country(gowalla_homes)

brightkite_homes.head()

### Adding temporal information to the check-in data

To be able to separate check-ins temporally, we add to each check-in, the month and season it was made in. 

In [ ]:
def to_datetime(row):
    return datetime.strptime(row['checkin_time'], '%Y-%m-%dT%H:%M:%SZ')

def get_month(row):
    """ Returns the string found in the row to the correctly formatted datetime object """
    return to_datetime(row).month

# Define the datetime intervals according to every season
Y = 2000 # Leap year to allow input X-02-29, which allows for leap days
seasons = [('winter', (datetime(Y,  1,  1),  datetime(Y,  3, 20))),
           ('spring', (datetime(Y,  3, 21),  datetime(Y,  6, 20))),
           ('summer', (datetime(Y,  6, 21),  datetime(Y,  9, 22))),
           ('autumn', (datetime(Y,  9, 23),  datetime(Y, 12, 20))),
           ('winter', (datetime(Y, 12, 21),  datetime(Y, 12, 31)))]

def get_season(row):
    """ 
        Returns the season from a certain row
        Adapted from https://stackoverflow.com/a/28688724 
    """
    date_time = to_datetime(row)
    date_time = date_time.replace(year=Y, hour=0, minute=0, second=0)
    
    return next(season for season, (start, end) in seasons if start <= date_time <= end)

In [ ]:
# Find the month associated with every check-in
checkin_brightkite['month'] = checkin_brightkite.parallel_apply(get_month, axis=1)
checkin_gowalla['month'] = checkin_gowalla.parallel_apply(get_month, axis=1)

# Find the season associated with every check-in
checkin_brightkite['season'] = checkin_brightkite.parallel_apply(get_season, axis=1)
checkin_gowalla['season'] = checkin_gowalla.parallel_apply(get_season, axis=1)

checkin_brightkite.head()

## Identify travelers

We now indentify users who travel. We define a traveler as a user who has made at least one check-in outside of his home country (determined by the country of its home location). 

In [ ]:
def traveler_check(group):
    """ A user is a traveler if he has ever checked in to more than one country """
    return True if len(set(group['cc'])) > 1 else False

In [ ]:
# Find travellers in both datasets
brightkite_homes['traveler'] = checkin_brightkite.groupby('user').parallel_apply(traveler_check)
gowalla_homes['traveler'] = checkin_gowalla.groupby('user').parallel_apply(traveler_check)

brightkite_homes.head()

## Pickle the data

In [ ]:
pickle.dump(checkin_brightkite, open('pickles/checkin_brightkite', 'wb'))
pickle.dump(checkin_gowalla, open('pickles/checkin_gowalla', 'wb'))

pickle.dump(brightkite_homes, open('pickles/brightkite_homes', 'wb'))
pickle.dump(gowalla_homes, open('pickles/gowalla_homes', 'wb'))